In [13]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import dlib

In [14]:
## Import landmark model
path = (r'..\shape_predictor_68_face_landmarks.dat')
predictor = dlib.shape_predictor(path)
detector = dlib.get_frontal_face_detector()

In [15]:
## Functions to get and annotate landmarks
def get_marks(image):
    rect = detector(image,1)
    if len(rect)>1:
        return 'error'
    if len(rect)==0:
        return 'error'
    return np.matrix([[p.x,p.y] for p in predictor(image,rect[0]).parts()])

def annotate_marks_on_image(image,landmarks):
    image_copy = image.copy()

    for index,point in enumerate(landmarks):
        position = (point[0,0],point[0,1])
        cv2.putText(image,str(index),position,fontFace= cv2.FONT_HERSHEY_COMPLEX,fontScale = 0.5,color = (0,255,0))
        cv2.circle(image,position,3,color = (0,255,0))
    return image

In [19]:
## Functions to extract top lip and bottom lip
def get_top_lip(landmarks):
    top = []
    for i in range(50,53):
        top.append(landmarks[i])
    for i in range(61,64):
        top.append(landmarks[i])
    top_lip_pts = np.squeeze(np.asarray(top))
    top_lip_mean = np.mean(top_lip_pts,axis=0)
    
    return int(top_lip_mean[1])

def get_bottom_lip(landmarks):
    bottom = []
    for i in range(65,68):
        bottom.append(landmarks[i])
    for i in range(56,59):
        bottom.append(landmarks[i])
    bottom_lip_pts = np.squeeze(np.asarray(bottom))
    bottom_lip_mean = np.mean(bottom_lip_pts,axis=0)
    
    return int(bottom_lip_mean[1])


In [17]:
## Return distance between lips to identify if the mouth is open
def open(image):
    landmarks = get_marks(image)

    if landmarks=='error':
        return image,0
    
    image_with_marks = annotate_marks_on_image(image,landmarks)
    top_lip_center = get_top_lip(landmarks)
    bottom_lip_center = get_bottom_lip(landmarks)
    distance = abs(top_lip_center - bottom_lip_center)
    return image_with_marks,distance

In [20]:
## Get the video stream
cap = cv2.VideoCapture(0)
yawn_counter = 0
status = False

while True:
    ret,frame = cap.read()
    image_marks,distance = open(frame)

    old_status = status

## Check if yawning or not based on distance
    if distance>20:
        status=True
        cv2.putText(frame,"Yawning",(50,400),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
        cv2.putText(frame,'Count: '+str(yawn_counter+1),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
    else:
        status=False
    
    if old_status==True and status == False:
        yawn_counter+=1
    
    cv2.imshow('Landmarks: ',image_marks)
    cv2.imshow('Yawn counts: ',frame)

    if cv2.waitKey(1)==13:
        break

cap.release()

cv2.destroyAllWindows()